<a href="https://colab.research.google.com/github/Kieranosaurus/HYPER/blob/master/MXNet_PGGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet
!pip install mxnet-cu101

import mxnet as mx
from mxnet import nd, gluon, init, autograd, gpu
from mxnet.gluon import nn
from mxnet.gluon.data.vision import datasets, transforms
from IPython import display
import matplotlib.pyplot as plt
import time
import numpy as np
import pickle
from PIL import Image
import matplotlib.image as mpimg
import os
from sklearn.linear_model import Ridge

In [2]:
from google.colab import drive
drive.mount('/content/drive')

!cp '/content/drive/MyDrive/Bachelor Thesis/HYPER/data_sub1_4096.dat' '/content/'
!cp '/content/drive/MyDrive/Bachelor Thesis/HYPER/pggan_weights' '/content/'
!cp '/content/drive/MyDrive/Bachelor Thesis/HYPER/ridge_model' '/content/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def load_weights(net, gain):
    with open('pggan_weights', 'rb') as f:
        wb = pickle.load(f)
    weights = []
    biases = []
    for i in range(0, 53, 2):
        weights.append(wb[i])
        biases.append(wb[i+1])

    names_biases = []
    for i in range(27):
        names_biases.append("net0_apply_bias"+str(i)+"_b")
    
    names_weights = []
    names_weights.append("net0_dense0_weight")
    for i in range(26):
        names_weights.append("net0_conv"+str(i)+"_weight")

    for i in range(0, len(weights)-1, 3):
        weights[i] = nd.transpose(nd.array(weights[i]))
        weights[i+1] = nd.transpose(nd.array(weights[i+1]))
        weights[i+2] = nd.transpose(nd.array(weights[i+2]))
        
        if i == 0:
            # The first dense layer has its gain divided by 4
            std = (gain / 4) / np.sqrt(weights[i].shape[1])
        else:
            std = gain / np.sqrt(np.prod([weights[i].shape[1], weights[i].shape[2], weights[i].shape[3]]))
        weights[i] = weights[i] * std

        std = gain / np.sqrt(np.prod([weights[i+1].shape[1], weights[i+1].shape[2], weights[i+1].shape[3]]))
        weights[i+1] = weights[i+1] * std

        # The gain for each toRGB layer is 1
        std = 1 / np.sqrt(np.prod([weights[i+2].shape[1], weights[i+2].shape[2], weights[i+2].shape[3]]))
        weights[i+2] = weights[i+2] * std

        net.collect_params()[names_weights[i]].set_data(weights[i])
        net.collect_params()[names_weights[i+1]].set_data(weights[i+1])
        net.collect_params()[names_weights[i+2]].set_data(weights[i+2])
        
        net.collect_params()[names_biases[i]].set_data(nd.array(biases[i]))
        net.collect_params()[names_biases[i+1]].set_data(nd.array(biases[i+1]))
        net.collect_params()[names_biases[i+2]].set_data(nd.array(biases[i+2]))

    return net

def face_from_latent(net, latents, path):
    outputs = []
    for i in range(latents.shape[0]):
        latent = nd.expand_dims(latents[i], 0)
        output = net(latent)
        outputs.append(output)
        # face = np.clip(np.rint((face.asnumpy() + 1.0) / 2.0 * 255.0), 0.0, 255.0).astype('uint8')
        # face = face.transpose((0, 3, 2, 1))

        # if not os.path.exists(path):
        #     os.mkdir(path)
        # save_path = os.path.join(path, '%d.png' % i)
        # Image.fromarray(face[0], 'RGB').save(save_path)

        # img = mpimg.imread(save_path)
        # plt.axis('off')
        # plt.imshow(img)
        # plt.show()
    
    return outputs

def get_latents(latentAmount, inputShape):
    latents = np.random.RandomState(2000).randn(latentAmount, inputShape)
    # selection = np.random.choice(1000, 10)
    # print("Selected latent vectors: ", selection)
    # latents = latents[selection]
    return latents

In [4]:
class apply_bias(gluon.HybridBlock):
    def __init__(self, shape):
        super(apply_bias, self).__init__()
        self.shape = shape
        with self.name_scope():
            self.b = self.params.get("b", init=init.Zero(), shape=shape)

    def hybrid_forward(self, F, x, b, *args, **kwargs):
        return F.broadcast_add(x, b.reshape([1, x.shape[1], 1, 1]))

class Net(gluon.HybridBlock):
    def __init__(self, **kwargs):
        super(Net, self).__init__(**kwargs)
        with self.name_scope():

            # self.lod_in = mx.sym.Variable("lod", init='np.float32')
            self.lod_in = np.float32(0.0)

            # Leaky ReLU Activation
            self.lrelu = nn.LeakyReLU(alpha=0.2)

            # Block 1
            self.dense = nn.Dense(units=8192, use_bias=False)
            self.bias1a = apply_bias((512,))
            self.conv1 = nn.Conv2D(channels=512, kernel_size=3, padding=(1,1), layout='NCHW', use_bias=False)
            self.bias1b = apply_bias((512,))
            # to RGB
            self.torgb1 = nn.Conv2D(in_channels=512, channels=3, kernel_size=1, layout='NCHW', use_bias=False)
            self.biasrgb1 = apply_bias((3,))

            # Block 2
            self.conv2a = nn.Conv2D(channels=512, kernel_size=3, padding=(1,1), layout='NCHW', use_bias=False)
            self.bias2a = apply_bias((512,))
            self.conv2b = nn.Conv2D(channels=512, kernel_size=3, padding=(1,1), layout='NCHW', use_bias=False)
            self.bias2b = apply_bias((512,))

            self.torgb2 = nn.Conv2D(in_channels=512, channels=3, kernel_size=1, layout='NCHW', use_bias=False)
            self.biasrgb2 = apply_bias((3,))

            # Block 3
            self.conv3a = nn.Conv2D(channels=512, kernel_size=3, padding=(1,1), layout='NCHW', use_bias=False)
            self.bias3a = apply_bias((512,))
            self.conv3b = nn.Conv2D(channels=512, kernel_size=3, padding=(1,1), layout='NCHW', use_bias=False)
            self.bias3b = apply_bias((512,))

            self.torgb3 = nn.Conv2D(in_channels=512, channels=3, kernel_size=1, layout='NCHW', use_bias=False)
            self.biasrgb3 = apply_bias((3,))

            # Block 4
            self.conv4a = nn.Conv2D(channels=512, kernel_size=3, padding=(1,1), layout='NCHW', use_bias=False)
            self.bias4a = apply_bias((512,))
            self.conv4b = nn.Conv2D(channels=512, kernel_size=3, padding=(1,1), layout='NCHW', use_bias=False)
            self.bias4b = apply_bias((512,))

            self.torgb4 = nn.Conv2D(in_channels=512, channels=3, kernel_size=1, layout='NCHW', use_bias=False)
            self.biasrgb4 = apply_bias((3,))

            # Block 5
            self.conv5a = nn.Conv2D(channels=256, kernel_size=3, padding=(1,1), layout='NCHW', use_bias=False)
            self.bias5a = apply_bias((256,))
            self.conv5b = nn.Conv2D(in_channels=256, channels=256, kernel_size=3, padding=(1,1), layout='NCHW', use_bias=False)
            self.bias5b = apply_bias((256,))

            self.torgb5 = nn.Conv2D(in_channels=256, channels=3, kernel_size=1, layout='NCHW', use_bias=False)
            self.biasrgb5 = apply_bias((3,))

            # Block 6
            self.conv6a = nn.Conv2D(channels=128, kernel_size=3, padding=(1,1), layout='NCHW', use_bias=False)
            self.bias6a = apply_bias((128,))
            self.conv6b = nn.Conv2D(in_channels=128, channels=128, kernel_size=3, padding=(1,1), layout='NCHW', use_bias=False)
            self.bias6b = apply_bias((128,))

            self.torgb6 = nn.Conv2D(in_channels=128, channels=3, kernel_size=1, layout='NCHW', use_bias=False)
            self.biasrgb6 = apply_bias((3,))

            # Block 7
            self.conv7a = nn.Conv2D(channels=64, kernel_size=3, padding=(1,1), layout='NCHW', use_bias=False)
            self.bias7a = apply_bias((64,))
            self.conv7b = nn.Conv2D(in_channels=64, channels=64, kernel_size=3, padding=(1,1), layout='NCHW', use_bias=False)
            self.bias7b = apply_bias((64,))

            self.torgb7 = nn.Conv2D(in_channels=64, channels=3, kernel_size=1, layout='NCHW', use_bias=False)
            self.biasrgb7 = apply_bias((3,))

            # Block 8
            self.conv8a = nn.Conv2D(channels=32, kernel_size=3, padding=(1,1), layout='NCHW', use_bias=False)
            self.bias8a = apply_bias((32,))
            self.conv8b = nn.Conv2D(in_channels=32, channels=32, kernel_size=3, padding=(1,1), layout='NCHW', use_bias=False)
            self.bias8b = apply_bias((32,))

            self.torgb8 = nn.Conv2D(in_channels=32, channels=3, kernel_size=1, layout='NCHW', use_bias=False)
            self.biasrgb8 = apply_bias((3,))

            # Block 9
            self.conv9a = nn.Conv2D(channels=16, kernel_size=3, padding=(1,1), layout='NCHW', use_bias=False)
            self.bias9a = apply_bias((16,))
            self.conv9b = nn.Conv2D(in_channels=16, channels=16, kernel_size=3, padding=(1,1), layout='NCHW', use_bias=False)
            self.bias9b = apply_bias((16,))

            self.torgb9 = nn.Conv2D(in_channels=16, channels=3, kernel_size=1, layout='NCHW', use_bias=False)
            self.biasrgb9 = apply_bias((3,))

    def pixel_norm(self, x, epsilon=1e-8):
        return x * nd.rsqrt(nd.mean(nd.square(x), axis=1, keepdims=True) + epsilon)

    def upscale_2d(self, x, factor=2):
        s = x.shape
        x = nd.reshape(x, [-1, s[1], s[2], 1, s[3], 1])
        x = nd.tile(x, [1, 1, 1, factor, 1, factor])
        x = nd.reshape(x, [-1, s[1], s[2] * factor, s[3] * factor])
        return x

    def lerp_clip(self, a, b, t): 
        return a + (b - a) * nd.clip(nd.array(t), 0.0, 1.0)

    def hybrid_forward(self, F, x):
        # Block 1                                        
        x = self.dense(self.pixel_norm(x))                              #184
        x = x.reshape([-1, 512, 4, 4])                                  #185
        x = self.pixel_norm(self.lrelu(self.bias1a(x)))                 #186
        x = self.pixel_norm(self.lrelu(self.bias1b(self.conv1(x))))     #188

        # Block 2
        x = self.upscale_2d(x)                                          #194
        x = self.pixel_norm(self.lrelu(self.bias2a(self.conv2a(x))))    #196
        x = self.pixel_norm(self.lrelu(self.bias2b(self.conv2b(x))))    #198

        # Block 3
        x = self.upscale_2d(x)                                          #194
        x = self.pixel_norm(self.lrelu(self.bias3a(self.conv3a(x))))    #196
        x = self.pixel_norm(self.lrelu(self.bias3b(self.conv3b(x))))    #198
        return x

        # # Block 4
        # x = self.upscale_2d(x)                                          #194
        # x = self.pixel_norm(self.lrelu(self.bias4a(self.conv4a(x))))    #196
        # x = self.pixel_norm(self.lrelu(self.bias4b(self.conv4b(x))))    #198

        # # Block 5
        # x = self.upscale_2d(x)                                          #194
        # x = self.pixel_norm(self.lrelu(self.bias5a(self.conv5a(x))))    #196
        # x = self.pixel_norm(self.lrelu(self.bias5b(self.conv5b(x))))    #198

        # # Block 6
        # x = self.upscale_2d(x)                                          #194
        # x = self.pixel_norm(self.lrelu(self.bias6a(self.conv6a(x))))    #196
        # x = self.pixel_norm(self.lrelu(self.bias6b(self.conv6b(x))))    #198

        # # Block 7
        # x = self.upscale_2d(x)                                          #194
        # x = self.pixel_norm(self.lrelu(self.bias7a(self.conv7a(x))))    #196
        # x = self.pixel_norm(self.lrelu(self.bias7b(self.conv7b(x))))    #198

        # # Block 8
        # x = self.upscale_2d(x)                                          #194
        # x = self.pixel_norm(self.lrelu(self.bias8a(self.conv8a(x))))    #196
        # x = self.pixel_norm(self.lrelu(self.bias8b(self.conv8b(x))))    #198

        # # Block 9
        # x = self.upscale_2d(x)                                          #194
        # x = self.pixel_norm(self.lrelu(self.bias9a(self.conv9a(x))))    #196
        # x = self.pixel_norm(self.lrelu(self.bias9b(self.conv9b(x))))    #198
        # # to RGB
        # x = self.biasrgb9(self.torgb9(x))                               #212

        # return x

In [5]:
# Define and initialise the network
net = Net()
net.initialize(ctx=gpu(0))

In [6]:
# Get latents from test data stimuli
with open('data_sub1_4096.dat', 'rb') as fp:
    X_train, T_train, X_test, T_test = pickle.load(fp) 

In [7]:
# Define latents as stimuli data
# latents = get_latents(2000, 512)
# inputs = nd.array(latents, ctx=gpu(0))
inputs = nd.array(T_train, ctx=gpu(0))

# Perform one forward pass to initialise all weights
net.forward(nd.expand_dims(inputs[0], 0))

# Load in the weight and bias parameters from the pre-trained model
gain = np.sqrt(2)
net = load_weights(net, gain) 

# Run the model on the latent values
outputs = face_from_latent(net, inputs, '/content/faces_mxnet/')

In [8]:
print(len(outputs))
print(outputs[0].shape)

1050
(1, 512, 16, 16)


In [9]:
training = outputs
targets = X_train
for i in range(len(training)):
    training[i] = training[i].asnumpy().flatten()

In [10]:
print(len(training))
print(len(targets))

print(training[0].shape)
print(targets[0].shape)

1050
1050
(131072,)
(4096,)


In [ ]:
linModel = Ridge()
linModel.fit(training, targets)

In [ ]:
inputs = nd.array(T_test, ctx=gpu(0))

outputs = face_from_latent(net, inputs, '/content/faces_mxnet/')

In [ ]:
training = outputs
targets = X_test
for i in range(len(training)):
    training[i] = training[i].asnumpy().flatten()

In [ ]:
linModel.score(training, targets)

In [11]:
voxel_weights = linModel.coef_
voxel_weights = np.reshape(voxel_weights, (-1,512,4,4))

In [12]:
average_voxel_weights = []
for i in voxel_weights:
    average_voxel_weights.append(np.mean(i, axis=0))
average_voxel_weights[0].shape

(4, 4)

In [13]:
# with open('ridge_model', 'wb') as f:
#     pickle.dump(linModel, f)
# !cp '/content/ridge_model' '/content/drive/MyDrive/Bachelor Thesis/HYPER/ridge_model'